# 1.Colab 환경 설정


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=688960 sha256=339b5442cb78bee69fd5a1317e9c1c7f8ebbe9399aa006ea43e0770d5890e7fa
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-owqsqg5s
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-owqsqg5s
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 2.데이터셋 전처리

In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Train

In [ ]:
import pandas as pd
chatbot_data = pd.read_csv('/content/drive/MyDrive/kobert/keyword_emotion_sentence_only_train.csv')

In [ ]:
chatbot_data.sample(n=10)

,Unnamed: 0,상황키워드,감정_대분류,사람문장
39493,47339,"대인관계(부부, 자녀)",슬픔,월급이 적다고 타박하는 아내한테 화를 내고 나와 버려서 너무 후회돼. 나는 나름 열...
17617,20764,"대인관계(부부, 자녀)",슬픔,자식들은 이제 나를 우러러보지 않는 것 같아. 예전에는 내가 가장 자랑스럽고 존경스...
33912,40391,"건강,죽음",불안,허리를 삐끗해서 몸을 움직이는 게 쉽지가 않네. 이따가 약속이 있는데 걱정이야. 오...
20128,23689,"대인관계(부부, 자녀)",불안,오랫동안 연락이 끊겼던 희진이가 오랜만에 전화를 했어. 너무 혼란스럽네. 당황스럽기...
30406,36026,학업 및 진로,슬픔,공부를 열심히 했는데 테스트를 망쳤어. 그렇지만 난 자신이 없는걸 난 정말 무식한가...
8545,10039,"연애,결혼,출산",분노,도저히 내 남자 친구를 이해할 수 없어. 데이트 때마다 모자에 슬리퍼만 신고 나와....
21171,24942,재정,기쁨,일하는 곳에서 고생이 많았다며 별도로 보너스를 챙겨줘서 감사한 마음이야. 그 동안 ...
18921,22230,"진로,취업,직장",상처,대기업 정직원이었던 내가 퇴사 후 중소기업의 면접에 모두 떨어져서 상처 받았고 충격...
41482,49853,대인관계,슬픔,맨날 야근해서 아들 얼굴 보기도 힘들어. 근데 아들이 집에 오면 반겨주지도 않네. ...
2353,2803,건강,기쁨,그동안 고혈압 때문에 고생도 하고 고민도 많이 했는데 혈압이 정상 수치를 되찾았다고...


In [ ]:
chatbot_data = chatbot_data.rename(columns={'감정_대분류': 'Emotion', '사람문장': 'Sentence'})

In [ ]:
chatbot_data

,Unnamed: 0,상황키워드,Emotion,Sentence
0,0,"진로,취업,직장",분노,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...
1,1,"진로,취업,직장",분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....
2,2,"진로,취업,직장",분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,3,"진로,취업,직장",분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,4,"진로,취업,직장",분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...
...,...,...,...,...
42869,51625,재정,분노,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고....
42870,51626,재정,불안,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나. 마음...
42871,51627,재정,상처,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정...
42872,51628,대인관계,불안,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가...


In [ ]:
chatbot_data['Emotion'].unique()

array(['분노', '기쁨', '불안', '슬픔', '상처'], dtype=object)

In [ ]:
chatbot_data.loc[(chatbot_data['Emotion'] == "불안"), 'Emotion'] = 0  #불안 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 1  #분노 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "상처"), 'Emotion'] = 2  #상처 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "기쁨"), 'Emotion'] = 4  #기쁨 => 4

data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고. .', '1']
['이번에 친구 녀석이 아홉 살 연하랑 결혼한다네. 난 아직 싱글인데 부러워 죽겠네. 부러운 정도가 아니라 엄청나게 질투 나지. 심지어 예쁘기까지 하던데. .', '2']
['나는 인스타그램같은 거 하기 싫은데 거기에 심취한 친구가 나한테도 하라고 강요해. 어디 같이 갈 때 그곳에 나도 태그하고 싶대. 나만 태그할 수 없대. 모르겠어. 친구는 그냥 나를 이해하지 못하는 것 같아.', '1']
['이번 주말에 집에서 뒹굴 거리며 푹 쉬려고 했는데 갑자기 회사에서 출근하래. 심지어 월요일부터 금요일까지 야근했거든? 이번 주 너무 피곤한데 어떡하나. 아니. 격주로 돌아가면서 근무하는 걸로 알고 있어.', '1']
['팀원이 업무에 실수가 잦아서 힘들어. 내가 관리자라 팀원의 실수는 내 몫이야. 팀원에게 조언을 해주었지.', '2']
['나는 운동도 성실히 한 사람이었는데 세월 앞에 다 의미 없는 것 같아. 곧 나도 죽고 말겠지? 내 여력으로 죽음을 피할 순 없으니 말야. 그래. 나한테 살 날이 얼마 안 남은 건 사실이니까.', '3']
['남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사라져. 사람들을 만나는 것이 어려워. 자꾸 사람들을 의심하게만 되고 말이야. 사람들을 볼 때 의심하고 불신하는 마음을 억눌러야겠어. 사람들을 색안경을 끼고 보지 않게.', '2']


In [ ]:
dataset_train = data_list

# test

In [ ]:
import pandas as pd
chatbot_data = pd.read_csv('/content/drive/MyDrive/kobert/keyword_emotion_sentence_only_validation.csv')

In [ ]:
chatbot_data = chatbot_data.rename(columns={'감정_대분류': 'Emotion', '사람문장': 'Sentence'})


In [ ]:
chatbot_data.loc[(chatbot_data['Emotion'] == "불안"), 'Emotion'] = 0  #불안 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 1  #분노 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "상처"), 'Emotion'] = 2  #상처 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "기쁨"), 'Emotion'] = 4  #기쁨 => 4

data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
dataset_test = data_list

# 3.Train data & Test data


In [ ]:
print(len(dataset_train))
print(len(dataset_test))

42874
5593


# 4.KoBERT 입력 데이터로 만들기

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# 5.KoBERT 학습모델 만들기


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

# 6.KoBERT 모델 학습시키기


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-27-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/670 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.7196192741394043 train acc 0.171875
epoch 1 batch id 201 loss 0.8017446398735046 train acc 0.37010261194029853
epoch 1 batch id 401 loss 0.5359920859336853 train acc 0.5196773690773068
epoch 1 batch id 601 loss 0.7492082118988037 train acc 0.5543105241264559
epoch 1 train acc 0.5764290079773545


<ipython-input-27-480b6a139979>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/88 [00:00<?, ?it/s]

...

epoch 4 test acc 0.48505681818181823


  0%|          | 0/670 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 1.1609784364700317 train acc 0.625
epoch 5 batch id 201 loss 0.22019954025745392 train acc 0.7805503731343284
epoch 5 batch id 401 loss 0.1370285302400589 train acc 0.8029925187032418
epoch 5 batch id 601 loss 0.9175103902816772 train acc 0.7697327371048253
epoch 5 train acc 0.7515391791044777


  0%|          | 0/88 [00:00<?, ?it/s]

epoch 5 test acc 0.7969318181818181


# 7.새로운 문장 테스트


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("불안이")
            elif np.argmax(logits) == 1:
                test_eval.append("분노가")
            elif np.argmax(logits) == 2:
                test_eval.append("상처가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("기쁨이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 집에 가려면 2시간 걸려
>> 입력하신 내용에서 불안이 느껴집니다.


하고싶은 말을 입력해주세요 : 집에 가고싶어
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 학원에 오기 싫어
>> 입력하신 내용에서 불안이 느껴집니다.


하고싶은 말을 입력해주세요 : 친구들이 나를 배신하고 놀러갔어
>> 입력하신 내용에서 상처가 느껴집니다.


하고싶은 말을 입력해주세요 : 나만 아이스크림이 없어
>> 입력하신 내용에서 상처가 느껴집니다.


하고싶은 말을 입력해주세요 : 올때 메로나
>> 입력하신 내용에서 상처가 느껴집니다.


하고싶은 말을 입력해주세요 : 올때 메로나 사와
>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : 쌍쌍바는 어때?
>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : 드립치지마
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 이호진 미친놈아
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 홍대가려면 어디로 가야하나요?
>> 입력하신 내용에서 불안이 느껴집니다.


하고싶은 말을 입력해주세요 : 뉴진스의 하입보이요
>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : 꽃 버트 어떄?
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 꽃버트 어떄?
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 꽃버트
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 꽃을 받았습니다
>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : 꽃 bert
>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : hello
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : WTF
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은

In [ ]:
import torch
import h5py

# 모델을 저장할 경로를 지정합니다.
PATH = "/content/drive/MyDrive/kobert/kobert_emotion_epoch5.h5"

# 모델의 state_dict를 저장합니다.
torch.save(model.state_dict(), PATH)

# h5py를 사용하여 .h5 파일로 저장합니다.
with h5py.File(PATH, "w") as f:
    for key in model.state_dict():
        f.create_dataset(key, data=model.state_dict()[key].cpu().numpy())

In [ ]:
import torch
import h5py
import numpy as np
# 모델 아키텍처를 정의합니다.
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# 모델을 로드할 경로를 지정합니다.
PATH = "/content/drive/MyDrive/kobert/kobert_emotion_epoch5.h5"
# .h5 파일을 열어서 state_dict를 로드합니다.
with h5py.File(PATH, "r") as f:
    for key in f.keys():
        param = torch.from_numpy(np.array(f[key]))
        model.state_dict()[key].copy_(param)

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")